## Experiment Setup

### Algorithm
    - Scheduling time T: 1 (in block-time e.g once per day if block arrival is once per day)
    - Scheduling policy: FCFS
    - No budget unlocking: data-lifetime=0.1, n=1 (At time 0.1 block gets fully unlocked)       

### Workload
    - Total number of blocks: 600 (initial blocks=1)
    - One task per block-unit
    - Blocks arrival time: 1 (e.g. 1 day)
    - Number of blocks: all previous blocks
    - Tasks lifetime: 1 (block-time e.g. 1 day = after that the task can't run)
    - Budget demand (epsilon): 1 (fixed for all incoming tasks for now)
    - Block budget initial capacity: 10
    - Number of tasks: (See task arrival time below)
    - Number of distinct queries: 1
    - Query types: Count only
    - RDP
   
### Cost Model for dynamic programming planner as of now
    - Minimizing aggregations + binary structure constraint

In [ ]:
from notebooks.caching.utils import analyze_experiment
tasks_path = "~/privacypacking/data/covid19/covid19_workload/privacy_tasks.csv"

In [ ]:
# analyze_experiment(tasks_path, "run_and_report_2022-11-09_17-25-58")

In [ ]:
# Cound of positive covid cases
# analyze_experiment(tasks_path, "run_and_report_2022-11-09_23-27-12")

In [ ]:
#  600 blocks

# Count of positive covid cases for black ppl who are between 0 and 18 years old
analyze_experiment(tasks_path, "run_and_report_2022-11-10_02-41-45/")

In [ ]:
#  600 blocks

# Count of positive covid cases for Indian ppl who are between 0 and 18 years old
analyze_experiment(tasks_path, "run_and_report_2022-11-10_02-58-42/")
